In [3]:
!pip install yfinance


  Using cached yfinance-0.2.66-py2.py3-none-any.whl.metadata (6.0 kB)
  Using cached multitasking-0.0.12-py3-none-any.whl
  Using cached peewee-3.18.3-cp313-cp313-macosx_11_0_arm64.whl
  Using cached curl_cffi-0.13.0-cp39-abi3-macosx_11_0_arm64.whl.metadata (13 kB)
  Using cached websockets-15.0.1-cp313-cp313-macosx_11_0_arm64.whl.metadata (6.8 kB)
Using cached yfinance-0.2.66-py2.py3-none-any.whl (123 kB)
Using cached curl_cffi-0.13.0-cp39-abi3-macosx_11_0_arm64.whl (3.0 MB)
Using cached websockets-15.0.1-cp313-cp313-macosx_11_0_arm64.whl (173 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [yfinance]3/5 [curl_cffi]


In [4]:
import yfinance as yf

In [5]:
import pandas as pd
import sqlite3

# 1️⃣ Define stock tickers
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META']

# 2️⃣ Pull live data from Yahoo Finance
data = yf.download(tickers, start="2024-01-01", end="2025-11-13")

# 3️⃣ Flatten data for database
data = data.stack(level=1).rename_axis(['Date', 'Ticker']).reset_index()

# 4️⃣ Save to local SQLite database
conn = sqlite3.connect("stock_data.db")
data.to_sql("stock_prices", conn, if_exists="replace", index=False)
conn.close()

print("✅ Stock data saved successfully at:")
print("/Users/snehabanerjee/Documents/Self Projects/Stock Market Intelligence Dashboard/stock_data.db")

/var/folders/rr/twk0045j4k9921sh957c7c500000gn/T/ipykernel_16213/3114644325.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start="2024-01-01", end="2025-11-13")
[*********************100%***********************]  5 of 5 completed

✅ Stock data saved successfully at:
/Users/snehabanerjee/Documents/Self Projects/Stock Market Intelligence Dashboard/stock_data.db



/var/folders/rr/twk0045j4k9921sh957c7c500000gn/T/ipykernel_16213/3114644325.py:11: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  data = data.stack(level=1).rename_axis(['Date', 'Ticker']).reset_index()


In [6]:

# 1️⃣ Connect to the database
conn = sqlite3.connect("stock_data.db")

# 2️⃣ Check table names
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("Tables in database:")
print(tables)

# 3️⃣ Preview the first 5 rows of the main table
df = pd.read_sql("SELECT * FROM stock_prices LIMIT 5;", conn)
df.head()

# 4️⃣ Close connection
conn.close()

Tables in database:
           name
0  stock_prices


In [7]:
print(df.columns)

Index(['Date', 'Ticker', 'Close', 'High', 'Low', 'Open', 'Volume'], dtype='object')


In [8]:
df.head(2)


,Date,Ticker,Close,High,Low,Open,Volume
0,2024-01-02 00:00:00,AAPL,183.903229,186.677036,182.169601,185.399096,82488700
1,2024-01-02 00:00:00,AMZN,149.929993,152.380005,148.389999,151.539993,47339400


In [9]:
df_pivot = df.pivot(index='Date', columns='Ticker', values='Close')
df_pivot.head(2)


Ticker,AAPL,AMZN,GOOGL,META,MSFT
Date,,,,,
2024-01-02 00:00:00,183.903229,149.929993,137.223526,344.168304,366.105499


In [10]:
conn.close()